# [Coworked with Jung0Jin in Tactics Lab](https://github.com/Jung0Jin)

In [1]:
# !pip install wget
import wget
wget.download('https://www.dropbox.com/s/7pn8swdrje4ei87/BMS_data.txt?dl=1')
wget.download('https://www.dropbox.com/s/4glojc5ya8h4hwk/excel_station_status.xlsx?dl=1')
wget.download('https://www.dropbox.com/s/r4cdabs9uopny5z/station_sequence_route.xls?dl=1')

100% [..........................................................................] 4031488 / 4031488

'station_sequence_route.xls'

In [2]:
import datetime
import pandas as pd
import networkx as nx

In [3]:
G = {}
input_file = open('BMS_data.txt','r')

init_line = input_file.readline()
data_line = input_file.readline()
data_line = data_line.split(',')
temp = 0
while True:
    temp+=1
    if temp%1000000==0:
        print(f'{temp}번째 줄')
    
    source_bus_id = data_line[3]
    source = data_line[4]
    source_time = datetime.datetime.strptime(data_line[5],'%Y%m%d%H%M%S')
    
    data_line = input_file.readline()
    if not data_line:break
        
    data_line = data_line.split(',')
    if data_line == ['']:
        data_line = input_file.readline()
        if not data_line:break
        data_line = data_line.split(',')
        continue
    target_bus_id = data_line[3]
    target = data_line[4]
    target_time = datetime.datetime.strptime(data_line[5],'%Y%m%d%H%M%S')

    if source_bus_id == target_bus_id:
        try:
            G[source,target][0]=G[source,target][0]+(target_time-source_time)
            G[source,target][1]+=1
        except KeyError:
            G[source,target] = [target_time-source_time,1]
        
for key in G:
    G[key] = G[key][0]/G[key][1]
print(len(G.values()))

1000000번째 줄
2000000번째 줄
3000000번째 줄
4000000번째 줄
5000000번째 줄
6000000번째 줄
7000000번째 줄
84023


In [4]:
## 정류소 간 시간이 0보다 작은 경우 삭제

for key in G.copy():
    if G[key] < datetime.timedelta(0):
        del G[key]
print(len(G.values()))

54212


In [5]:
stn_status = pd.read_excel('excel_station_status.xlsx',usecols=['정류소 ID','정류소 번호','ISC 정류소 ID','위도','경도'],
             dtype=str)
stn_status.head()

,정류소 번호,정류소 ID,ISC 정류소 ID,위도,경도
0,37319,163000319,2802757,126.68018830433286,37.457360395358165
1,37326,163000326,2802755,126.67992703811578,37.45776642315442
2,42228,168000228,2800619,126.67339452968905,37.52184277702059
3,40158,166000158,2801688,126.71126751729724,37.490948333960574
4,89146,168001146,2806735,126.61697759175779,37.559185515339


In [14]:
mapping = {stn_status.loc[i,'정류소 ID']:stn_status.loc[i,'ISC 정류소 ID'] for i in stn_status.index}

In [15]:
id_list = stn_status['정류소 ID'].tolist()
isc_id_list = stn_status['ISC 정류소 ID'].tolist()
for key in G.copy():
    source,target = key
    if (source not in id_list) or (target not in id_list):
        del G[key]
        continue
    source_ = mapping[source]
    target_ = mapping[target]
    
    G[(source_,target_)] = G[key].total_seconds()
    del G[key]

In [16]:
networkx_G = nx.DiGraph()
for key,value in G.items():
    networkx_G.add_edge(*key,time=value)

확인 및 실험(511번)

In [17]:
route_sequence = pd.read_excel('station_sequence_route.xls',dtype=str)
sequence_511 = route_sequence[route_sequence['노선번호']=='511']
sequence_511

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,정류소명,정류소표준ID,정류소단축ID,노선ID,노선번호,정류소순번,버스유형
17224,독배로317번길(삼거리),163000634,37634,165000073,511,1,지선
17225,SK스카이뷰104동,163000587,37587,165000073,511,2,지선
17226,SK스카이뷰109동,163000607,37607,165000073,511,3,지선
17227,굴다리,163000142,37142,165000073,511,4,지선
17228,용현고가교,163000611,37611,165000073,511,5,지선
17229,인하대후문,163000165,37165,165000073,511,6,지선
17230,정석항공과학고,163000168,37168,165000073,511,7,지선
17231,학산소극장,163000169,37169,165000073,511,8,지선
17232,용남파출소,163000177,37177,165000073,511,9,지선
17233,용일사거리,163000208,37208,165000073,511,10,지선


In [18]:
sequence_511['ISC 정류소 ID'] = None
for idx in sequence_511.index:
    id = sequence_511.loc[idx,'정류소표준ID']
    isc_id = mapping[id]
    sequence_511.loc[idx,'ISC 정류소 ID'] = isc_id
sequence_511_isc = sequence_511['ISC 정류소 ID'].tolist()
sequence_511_isc

C:\Users\skatl\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\skatl\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


['2807441',
 '2806808',
 '2806914',
 '2802920',
 '2807167',
 '2802898',
 '2802895',
 '2802894',
 '2802887',
 '2802862',
 '2802803',
 '2802774',
 '2802760',
 '2802617',
 '2802733',
 '2807110',
 '2802710',
 '2802734',
 '2802756',
 '2805155',
 '2802794',
 '2802852',
 '2802880',
 '2802890',
 '2802893',
 '2802896',
 '2805500',
 '2802934',
 '2806811',
 '2806809']

In [19]:
total_time = 0
avg_time = sum(list(G.values()))/len(list(G.values()))
for source,target in zip(sequence_511_isc[:-1],sequence_511_isc[1:]):
    if target == sequence_511_isc[-1]:break
    try:
        time = networkx_G.edges[(source,target)]['time']
    except KeyError:
        time = avg_time
    total_time+=time
print(total_time)

ZeroDivisionError: division by zero

In [ ]:
total_time